In [9]:
from PIL import Image
from matplotlib.pyplot import imshow
import pandas
import matplotlib.pylab as plt
import os
import glob

In [10]:
def show_data(data_sample, shape = (28, 28)):
    plt.imshow(data_sample[0].numpy().reshape(shape), cmap='gray')
    plt.title('y = ' + data_sample[1])

In [11]:
directory="/Users/lizhang/Dropbox/Computational Tools/NLP_sentencetransformer/concrete_crack_images_for_classification"

In [12]:
negative='Negative'

In [13]:
negative_file_path=os.path.join(directory,negative)
negative_file_path

'/Users/lizhang/Dropbox/Computational Tools/NLP_sentencetransformer/concrete_crack_images_for_classification/Negative'

In [14]:
#look at the file names of the first three samples in the folder 
os.listdir(negative_file_path)[0:3]

['14147.jpg', '04354.jpg', '13628.jpg']

In [15]:
# join the directory path and file names from above to form full path to each images. Example
[os.path.join(negative_file_path,file) for file in  os.listdir(negative_file_path)][0:3]

['/Users/lizhang/Dropbox/Computational Tools/NLP_sentencetransformer/concrete_crack_images_for_classification/Negative/14147.jpg',
 '/Users/lizhang/Dropbox/Computational Tools/NLP_sentencetransformer/concrete_crack_images_for_classification/Negative/04354.jpg',
 '/Users/lizhang/Dropbox/Computational Tools/NLP_sentencetransformer/concrete_crack_images_for_classification/Negative/13628.jpg']

In [16]:
print("test.jpg".endswith(".jpg"))
print("test.mpg".endswith(".jpg"))

True
False


In [17]:
#find all the file paths to .jpg files and sort in ascending order
negative_files=[os.path.join(negative_file_path,file) for file in  os.listdir(negative_file_path) if file.endswith(".jpg")]
negative_files.sort()
negative_files[0:3]

['/Users/lizhang/Dropbox/Computational Tools/NLP_sentencetransformer/concrete_crack_images_for_classification/Negative/00001.jpg',
 '/Users/lizhang/Dropbox/Computational Tools/NLP_sentencetransformer/concrete_crack_images_for_classification/Negative/00002.jpg',
 '/Users/lizhang/Dropbox/Computational Tools/NLP_sentencetransformer/concrete_crack_images_for_classification/Negative/00003.jpg']

In [19]:
#repeat the above steps for positive directory
positive="Positive"
positive_file_path=os.path.join(directory,positive)
positive_files=[os.path.join(positive_file_path,file) for file in  os.listdir(positive_file_path) if file.endswith(".jpg")]
positive_files.sort()
positive_files[0:3]

['/Users/lizhang/Dropbox/Computational Tools/NLP_sentencetransformer/concrete_crack_images_for_classification/Positive/00001.jpg',
 '/Users/lizhang/Dropbox/Computational Tools/NLP_sentencetransformer/concrete_crack_images_for_classification/Positive/00002.jpg',
 '/Users/lizhang/Dropbox/Computational Tools/NLP_sentencetransformer/concrete_crack_images_for_classification/Positive/00003.jpg']

In [20]:
image1 = Image.open(negative_files[0])

In [35]:
len(negative_files)

20000

In [37]:
image.size

(227, 227)

In [38]:
number_of_samples=len(positive_files)+len(negative_files)

In [41]:
Y=torch.zeros([number_of_samples])

In [43]:
Y=Y.type(torch.LongTensor)
Y.type()

'torch.LongTensor'

In [52]:
#set the even elements to class one and the odd elements to class zero
#in other words Y contains the class labels for all images
Y[::2]=1
Y[1::2]=0
Y

tensor([1, 0, 1,  ..., 0, 1, 0])

In [49]:
# a list all_files was created, in which the even indexes point to the path to images with positive or cracked samples 
#and the odd indexes point to the negative images or images with out cracks.


all_files=[]
for i in range(0,20000):
    all_files.append(positive_files[i])
    all_files.append(negative_files[i])

In [51]:
len(all_files)

40000

In [129]:
from torch.utils.data import Dataset
class Dataset(Dataset):

    # Constructor
    def __init__(self,transform=None,train=True):
        directory="/Users/lizhang/Dropbox/Computational Tools/NLP_sentencetransformer/concrete_crack_images_for_classification"
        positive="Positive"
        negative="Negative"

        positive_file_path=os.path.join(directory,positive)
        negative_file_path=os.path.join(directory,negative)
        positive_files=[os.path.join(positive_file_path,file) for file in  os.listdir(positive_file_path) if file.endswith(".jpg")]
        positive_files.sort()
        negative_files=[os.path.join(negative_file_path,file) for file in  os.listdir(negative_file_path) if file.endswith(".jpg")]
        negative_files.sort()

        self.all_files=[None]*number_of_samples
        self.all_files[::2]=positive_files
        self.all_files[1::2]=negative_files 
        # The transform is goint to be used on image
        self.transform = transform
        #torch.LongTensor
        self.Y=torch.zeros([number_of_samples]).type(torch.LongTensor)
        self.Y[::2]=1
        self.Y[1::2]=0
        
        if train:
            self.all_files=all_files[0:30000]
            self.Y=Y[0:30000]
            self.len=len(self.all_files)
        else:
            self.all_files=all_files[30000:]
            self.Y=Y[30000:]
            self.len=len(self.all_files)
    # Get the length
    def __len__(self):
        return self.len
    
    # Getter
    def __getitem__(self, idx):
        
        
        image=Image.open(self.all_files[idx])
        y=self.Y[idx]
          
        
        # If there is any transform method, apply it onto the image
        if self.transform:
            image = self.transform(image)

        return image, y

In [116]:
from PIL import Image
import matplotlib.pyplot as plt
import os
import glob
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn as nn
from torch import optim 

In [130]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
transform =transforms.Compose([ transforms.ToTensor(), transforms.Normalize(mean, std)])

In [180]:
#create 
dataset_train=Dataset(transform=transform,train=True)
dataset_val=Dataset(transform=transform,train=False)

In [181]:
dataset_train[0][0].shape

torch.Size([3, 227, 227])

In [182]:
size_of_image=3*227*227
size_of_image

154587

In [134]:
torch.manual_seed(0)

In [183]:
trainloader = DataLoader(dataset=dataset_train, batch_size=1000)
valloader = DataLoader(dataset=dataset_val, batch_size=1000)

In [191]:
class Softmax(nn.Module):
    def __init__(self):
        super(Softmax, self).__init__()
        self.linear=nn.Linear(size_of_image, 2)
    
    def forward(self,x):
        x=x.view(-1, size_of_image)
        out=self.linear(x)
        return(out)

In [192]:
model=Softmax()

In [193]:
criterion= nn.CrossEntropyLoss()

In [194]:
learning_rate=0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [195]:
n_epochs=5
cost_list=[]
accuracy_list=[]
N_test=len(dataset_val)
COST=0

def train_model(n_epochs):
    for epoch in range(n_epochs):
        COST=0
        for x, y in trainloader:
            optimizer.zero_grad()
            z = model(x)
            loss = criterion(z, y)
            loss.backward()
            optimizer.step()
            COST+=loss.data
        
        cost_list.append(COST)
        correct=0
        #perform a prediction on the validation  data  
        for x_test, y_test in valloader:
            z = model(x_test)
            _, yhat = torch.max(z.data, 1)
            correct += (yhat == y_test).sum().item()
        accuracy = correct/ N_test
        accuracy_list.append(accuracy)
        print(epoch)

In [196]:
train_model(5)

0
1
2
3
4


In [184]:
trainloader

In [198]:
accuracy_list

[0.6828, 0.5419, 0.75, 0.759, 0.7534]

In [ ]:
import torch
import torchvision.models as models
model=models.resnet18(pretrained=True)
